In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/stockmarket-sentiment-dataset/stock_data.csv")

In [ ]:
import nltk
from nltk.corpus import stopwords
import textblob
from textblob import TextBlob
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing  import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
import re

In [ ]:
data

Tr - Veri setine ilk bakış

Eng - First look at the dataset

In [ ]:
data.info()

In [ ]:
data["Sentiment"].value_counts()

Tr - Veri ön işleme

Eng - Data preprocessing

In [ ]:
data["Sentiment"] = data["Sentiment"].replace(-1,0)

In [ ]:
sw = stopwords.words('english')

In [ ]:
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(data["Text"])

In [ ]:
data.head(5)

Tr- Basit bir özellik çıkarımı. Text verilerimizi tokenleştirip kelimeleri bir integer olarak atıyoruz. Sonra modele vereceğimiz cümlelerimizin boyutunu belirliyoruz. Bu önemlidir, çünkü çok fazla boyutlu kelimeler modelimizi yorabilir veya az geçen, bir öneme sahip olmaya kelimelerimizide vermiş olabiliriz.

Eng- Simple feature extraction. We specify our text data and throw out the words as whole numbers. Then we determine the size of our sentences that we will give to the model.This is important because we can tire our model of words with too many dimensions, or we may have given meaningless words to the model.

In [ ]:
data_tokens = tokenizer.texts_to_sequences(data["Text"])

In [ ]:
data_tokens[15]

In [ ]:
data["Text"][15]

Tr - Buraya kadar tokenleştirme işlemini yaptık. Burdan sonra ise özellik çıkarımı yaparak kelimelerimizi hem vektörleştiriyoruz, hemde dediğimiz gibi modelimize vereceğimiz cümlelerin boyutunu beliyorluyoruz.

Eng- After that, we do feature extraction. We both vectorize and, as I said, determine the sentences we will give to our model.

In [ ]:
num_tokens = [len(tokens) for tokens in data_tokens]

In [ ]:
num_tokens = np.array(num_tokens)

In [ ]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

Tr - Veri setimizde 26'dan fazla kelimeden oluşan cümlelerimizin veri setimize oranı sadece %7.

Eng - The ratio of our sentences consisting of more than 26 words in the data set to our data set is only %7.

In [ ]:
data_pad = pad_sequences(data_tokens, maxlen = max_tokens)

In [ ]:
data_pad.shape

Tr - Model Eğitimi

Eng - Model Training

In [ ]:
train_sequences, test_sequences, y_train, y_test = train_test_split(data_pad, data["Sentiment"], train_size=0.8, shuffle=True, random_state=1)

Tr - Modelimizi oluşutuyoruz.

Eng - We create our model.

In [ ]:
model = Sequential()

Tr - Kelime matrisimizi oluşturuyoruz. Ben deneyerek en başarılı paramtreleri bu şekilde buldum.

Eng - We create our word matrix. This is how I found the most successful parameters by trying.

In [ ]:
model.add(Embedding(input_dim = 50000,
                    output_dim = 300,
                   input_length = max_tokens,
                   name = "embeding_layer"))

In [ ]:
model.add(GRU(units = 64, return_sequences = True))
model.add(GRU(units = 32, return_sequences = True))
model.add(GRU(units = 16, return_sequences = True))
model.add(GRU(units = 8, return_sequences = True))
model.add(GRU(units = 4, return_sequences = False))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
optimizer = Adam(lr = 3e-5)

In [ ]:
model.compile(loss = 'binary_crossentropy',
              optimizer = optimizer,
              metrics = ["accuracy"])

In [ ]:
model.summary()

In [ ]:
model.fit(train_sequences, y_train, epochs = 20, batch_size = 32)

In [ ]:
results = model.evaluate(test_sequences, y_test, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))